In [9]:
import pandas as pd
import numpy as np
# from sklearn.linear_model import Ridge
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import xgboost as xgb
import re

### I Méthode Holdout

In [10]:
list(df_for_cv_used)

['CN_school',
 'CO_day',
 'CO_holiday',
 'CO_month',
 'CO_season',
 'CO_weather',
 'CO_workingday',
 'NN_soleil',
 'NN_tendance_1',
 'NN_tendance_2',
 'NO_atemp',
 'NO_humidity',
 'NO_temp',
 'NO_windspeed',
 'datetime',
 'CO_hour',
 'CO_year']

In [11]:
# On pourra aussi utiliser "df_for_cv_o_n.pkl"
# df_for_cv_interaction_and_dummies = pd.read_pickle("df_for_cv_interaction_and_dummies.pkl")

# OU 

df_for_cv_used = pd.read_pickle("../data/df_for_cv_o_n_sk_encoded.pkl")

all_var_x = [each for each in list(df_for_cv_used) if each != "count" and each != "datetime"]

# df_for_cv_interaction_and_dummies.shape
X_train = df_for_cv_used[all_var_x]
Y_train = df_for_cv_used[["count"]]

Exclusion des variables Quartiles et des variables "Special"

In [12]:
list(X_train)

['CN_school',
 'CO_day',
 'CO_holiday',
 'CO_month',
 'CO_season',
 'CO_weather',
 'CO_workingday',
 'NN_soleil',
 'NN_tendance_1',
 'NN_tendance_2',
 'NO_atemp',
 'NO_humidity',
 'NO_temp',
 'NO_windspeed',
 'CO_hour',
 'CO_year']

In [13]:
variables_to_keep = [each for each in list(X_train) if each[0] != 'S' and each[-1] != 'Q']

In [14]:
X_train = X_train[variables_to_keep]

### Petit processing sur le nom des features

In [15]:
features_names = list(X_train)
new_features_names = [re.sub("\[|\]|\<|\>",'',line) for line in features_names]
X_train.columns = new_features_names

## XGBoost valeurs par défaut

In [18]:
X_train.shape

(10886, 16)

In [17]:
# Calcul des Y = log(Y + 1)
Y_train['log_count'] = Y_train['count'].map(lambda x: np.log1p(x))
Y_train = Y_train.drop("count", axis = 1)

/home/osboxes/anaconda3/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Création "train" + "validation" = méthode Holdout

In [19]:
X_train.head()

,CN_school,CO_day,CO_holiday,CO_month,CO_season,CO_weather,CO_workingday,NN_soleil,NN_tendance_1,NN_tendance_2,NO_atemp,NO_humidity,NO_temp,NO_windspeed,CO_hour,CO_year
0,0,2,1,4,2,0,0,-68.036186,0.418093,0,14.395,81,9.84,0.0,0,0
1,0,2,1,4,2,0,0,-73.948688,0.418093,0,13.635,80,9.02,0.0,1,0
2,0,2,1,4,2,0,0,-71.066026,0.418093,0,13.635,80,9.02,0.0,12,0
3,0,2,1,4,2,0,0,-62.016308,0.418093,0,14.395,75,9.84,0.0,17,0
4,0,2,1,4,2,0,0,-50.973158,0.418093,0,14.395,75,9.84,0.0,18,0


In [20]:
X_train.shape

(10886, 16)

In [21]:
Y_train.shape

(10886, 1)

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train,
                                                    Y_train,
                                                    test_size = 0.3,
                                                    random_state = 0)

In [23]:
X_train.head()

,CN_school,CO_day,CO_holiday,CO_month,CO_season,CO_weather,CO_workingday,NN_soleil,NN_tendance_1,NN_tendance_2,NO_atemp,NO_humidity,NO_temp,NO_windspeed,CO_hour,CO_year
8620,1,6,0,1,0,0,1,68.660778,1.563340,578,37.120,52,32.80,11.0014,5,1
8985,1,4,0,1,0,0,1,22.375897,1.563340,593,35.605,36,32.80,0.0000,10,1
6171,0,1,0,3,2,0,1,9.692626,2.017690,408,6.820,50,4.92,12.9980,23,1
6473,0,5,0,7,2,0,1,-43.868511,2.446521,430,13.635,61,11.48,15.0013,16,1
10877,0,6,0,2,1,0,1,22.211826,1.617721,718,21.210,50,17.22,19.0012,7,1


In [24]:
X_test.shape

(3266, 16)

In [25]:
list(X_train)

['CN_school',
 'CO_day',
 'CO_holiday',
 'CO_month',
 'CO_season',
 'CO_weather',
 'CO_workingday',
 'NN_soleil',
 'NN_tendance_1',
 'NN_tendance_2',
 'NO_atemp',
 'NO_humidity',
 'NO_temp',
 'NO_windspeed',
 'CO_hour',
 'CO_year']

In [130]:
%%time
n_iter=85

# df_result_models = pd.DataFrame(columns = ['colsample_bytree','gamma','learning_rate','max_depth','n_estimators','subsample','reg_alpha','RMSE_train','RMSE_test'])
df_result_models = pd.DataFrame(columns = ['colsample_bytree','gamma','learning_rate','max_depth','n_estimators','subsample','reg_lambda','RMSE_train','RMSE_test'])

for each in range(1,n_iter):
    
    # print ("Modèle n°", each)
    colsample_bytree_value = np.random.uniform(0.3,0.7)
    gamma_value = np.random.uniform(0, 0.5)
    learning_rate_value = np.random.uniform(0.03, 0.3)
    max_depth_value = np.random.randint(2, 6)
    
    # n_estimators_value = np.random.randint(100, 150)
    n_estimators_value = np.random.randint(400, 500)
    subsample_value = np.random.uniform(0.4, 0.6)
    # Ajouter reg_alpha ou reg_lambda
    
    # reg_alpha_value = np.random.uniform(0, 0.05)
    # reg_alpha_value = np.random.uniform(0, 10)
    reg_lambda_value = np.random.uniform(0, 10)
    
    # print ("max_depth", max_depth_value)
    # print ("reg_alpha", reg_alpha_value)
    
    xgb_model = xgb.XGBRegressor(
        colsample_bytree = colsample_bytree_value,
        gamma = gamma_value,
        learning_rate = learning_rate_value,
        max_depth = max_depth_value,
        n_estimators = n_estimators_value,
        subsample = subsample_value,
        # reg_alpha = reg_alpha_value,
        reg_lambda = reg_lambda_value,
        objective="reg:squarederror")
        
        # eval_metric = "rmse"
          
    result = xgb_model.fit(X_train, Y_train)
    
    predicted_values = xgb_model.predict(X_train)
    # print ("predicted_values", predicted_values)
    predicted_values = np.expand_dims(predicted_values, axis=1)
        
    RMSE_train_value = np.sqrt(np.mean((predicted_values - Y_train.values)**2, axis=0))
    # print("le score RMLSE sur le training set vaut : {}".format(RMSE))
    
    predicted_values = xgb_model.predict(X_test)
    predicted_values = np.expand_dims(predicted_values, axis=1)
        
    RMSE_test_value = np.sqrt(np.mean((predicted_values - Y_test.values)**2, axis=0))
    # print("le score RMSE sur le test set vaut : {}".format(RMSE))
    
    if RMSE_train_value != None and RMSE_test_value != None:
    
        df_result_models = df_result_models.append({'colsample_bytree': colsample_bytree_value, 
                                 'gamma':gamma_value,
                                 'learning_rate':learning_rate_value,
                                 'max_depth':max_depth_value,
                                 'n_estimators':n_estimators_value,
                                 'subsample':subsample_value,
                                 # 'reg_alpha':reg_alpha_value,
                                 'reg_lambda':reg_lambda_value,
                                 'RMSE_train':RMSE_train_value,
                                 'RMSE_test':RMSE_test_value}, ignore_index= True)

CPU times: user 4min 40s, sys: 1.29 s, total: 4min 41s
Wall time: 4min 51s


In [124]:
# n_estimators entre 100 à 150 et régularisation L1 entre 0 et 10
df_result_models[df_result_models["RMSE_test"]  == df_result_models["RMSE_test"].min()[0]]

,colsample_bytree,gamma,learning_rate,max_depth,n_estimators,subsample,reg_alpha,RMSE_train,RMSE_test
29,0.561203,0.056888,0.11314,5,131,0.485035,0.672504,[0.24326636325942566],[0.298649468865716]


In [126]:
# n_estimators entre 400 à 500 et régularisation L1 entre 0 et 10
df_result_models[df_result_models["RMSE_test"]  == df_result_models["RMSE_test"].min()[0]]

,colsample_bytree,gamma,learning_rate,max_depth,n_estimators,subsample,reg_alpha,RMSE_train,RMSE_test
24,0.49319,0.192595,0.103625,5,440,0.543256,1.672217,[0.22553366470978184],[0.29043559328525825]


Cette fois avec régularisation L2 avec lambda : entre 0 et 10

In [129]:
# n_estimators entre 100 à 150 et régularisation L2 entre 0 et 10
df_result_models[df_result_models["RMSE_test"]  == df_result_models["RMSE_test"].min()[0]]

,colsample_bytree,gamma,learning_rate,max_depth,n_estimators,subsample,reg_lambda,RMSE_train,RMSE_test
58,0.527602,0.387389,0.129492,5,134,0.513007,9.068883,[0.2584567246348274],[0.3010008457426222]


In [131]:
# n_estimators entre 400 à 500 et régularisation L2 entre 0 et 10
df_result_models[df_result_models["RMSE_test"]  == df_result_models["RMSE_test"].min()[0]]

,colsample_bytree,gamma,learning_rate,max_depth,n_estimators,subsample,reg_lambda,RMSE_train,RMSE_test
39,0.675896,0.103326,0.088006,5,433,0.59952,6.282758,[0.1923565255813504],[0.2867988720875062]


On essaye avec 1000 arbres et Lambda: entre 5 et 20

In [28]:
%%time
xgb_model = xgb.XGBRegressor(
        colsample_bytree = 0.675896,
        gamma = 0.103326,
        learning_rate = 0.088006,
        max_depth = 5,
        n_estimators = 1000,
        subsample = 0.59952,
        # reg_alpha = reg_alpha_value,
        reg_lambda = np.random.uniform(5, 20),
        objective="reg:squarederror")
        
        # eval_metric = "rmse"
        
# print ("reg_lambda used:", reg_lambda)
          
result = xgb_model.fit(X_train, Y_train)

predicted_values = xgb_model.predict(X_train)
# print ("predicted_values", predicted_values)
predicted_values = np.expand_dims(predicted_values, axis=1)

RMSE_train_value = np.sqrt(np.mean((predicted_values - Y_train.values)**2, axis=0))
# print("le score RMLSE sur le training set vaut : {}".format(RMSE))

predicted_values = xgb_model.predict(X_test)
predicted_values = np.expand_dims(predicted_values, axis=1)

RMSE_test_value = np.sqrt(np.mean((predicted_values - Y_test.values)**2, axis=0))

CPU times: user 15.2 s, sys: 419 ms, total: 15.6 s
Wall time: 15.8 s


In [29]:
RMSE_train_value
# On avait : array([0.16629486])

array([0.15105345])

In [30]:
RMSE_test_value
# On avait : array([0.28462085])

array([0.28635527])

## SAUVEGARDE avec xgb_model

In [32]:
from sklearn.externals import joblib
# pour le save
joblib.dump(xgb_model, "xgb_0.408.pkl", compress=9)
# pour le reload
mymodel = joblib.load("xgb_0.408.pkl")

/home/osboxes/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['xgb_0.408.pkl']

In [ ]:
ON SOUMET A Kaggle !!!

In [ ]:
On essaye avec 5000 arbres et Lambda: entre 5 et 20

In [138]:
%%time
xgb_model = xgb.XGBRegressor(
        colsample_bytree = 0.675896,
        gamma = 0.103326,
        learning_rate = 0.088006,
        max_depth = 5,
        n_estimators = 5000,
        subsample = 0.59952,
        # reg_alpha = reg_alpha_value,
        reg_lambda = np.random.uniform(5, 20),
        objective="reg:squarederror")
        
        # eval_metric = "rmse"
          
result = xgb_model.fit(X_train, Y_train)

predicted_values = xgb_model.predict(X_train)
# print ("predicted_values", predicted_values)
predicted_values = np.expand_dims(predicted_values, axis=1)

RMSE_train_value = np.sqrt(np.mean((predicted_values - Y_train.values)**2, axis=0))
# print("le score RMLSE sur le training set vaut : {}".format(RMSE))

predicted_values = xgb_model.predict(X_test)
predicted_values = np.expand_dims(predicted_values, axis=1)

RMSE_test_value = np.sqrt(np.mean((predicted_values - Y_test.values)**2, axis=0))

CPU times: user 1min 6s, sys: 494 ms, total: 1min 7s
Wall time: 1min 10s


In [162]:
RMSE_train_value

array([0.16629486])

In [163]:
RMSE_test_value

array([0.28462085])

In [ ]:
Pas mieux qu'avec 1000 arbres et Lambda: entre 5 et 20

SOUMETTRE A KAGGLE !!!!

In [168]:
df_to_predict = pd.read_pickle("../data/df_test_kaggle_o_n_sk_encoded.pkl") 
df_to_predict.shape

(6493, 17)

In [170]:
df_to_predict.head()

,CN_school,CO_day,CO_holiday,CO_month,CO_season,CO_weather,CO_workingday,NN_soleil,NN_tendance_1,NN_tendance_2,NO_atemp,NO_humidity,NO_temp,NO_windspeed,datetime,CO_hour,CO_year
0,0,4,0,4,2,0,1,-64.731114,0.920373,19,11.365,56,10.66,26.0027,2011-01-20 00:00:00,0,0
1,0,4,0,4,2,0,1,-70.858731,0.920373,19,13.635,56,10.66,0.0000,2011-01-20 01:00:00,1,0
2,0,4,0,4,2,0,1,-69.313963,0.920373,19,13.635,56,10.66,0.0000,2011-01-20 02:00:00,12,0
3,0,4,0,4,2,0,1,-61.327846,0.920373,19,12.880,56,10.66,11.0014,2011-01-20 03:00:00,17,0
4,0,4,0,4,2,0,1,-50.733442,0.920373,19,12.880,56,10.66,11.0014,2011-01-20 04:00:00,18,0


In [161]:
# df_to_predict['CN_school'].sum()

2152

In [171]:
# Ajout de la prédiction au dataframe "df_to_predict"
df_temp = df_to_predict.drop("datetime", axis = 1)
features_names = list(df_temp)

# new_features_names = [re.sub("\[|\]|\<|\>",'',line) for line in features_names]
# df_temp.columns = new_features_names

log_pred = np.expand_dims(xgb_model.predict(df_temp), axis=1)

print (log_pred)

[[2.9426036]
 [1.8973216]
 [1.671432 ]
 ...
 [4.8414903]
 [4.4583635]
 [3.7573152]]


In [172]:
df_to_predict["count"] = list((np.exp(log_pred) - 1)[:,0])
df_to_predict[["datetime", "count"]].to_csv("../data/submission_to_send/pred_xgboost_regression_holdout_1000.csv", index= False)

In [ ]:
KAGGLE SCORE = 0.40892

### II Méthode randomized cross validation

In [74]:
# df_for_cv_interaction_and_dummies = pd.read_pickle("df_for_cv_o_n_sk_encoded.pkl")

# OU
df_for_cv_interaction_and_dummies = pd.read_pickle("df_for_cv_interaction_and_dummies.pkl")

all_var_x = [each for each in list(df_for_cv_interaction_and_dummies) if each != "count" and each != "datetime"]

# df_for_cv_interaction_and_dummies.shape
X_train = df_for_cv_interaction_and_dummies[all_var_x]
Y_train = df_for_cv_interaction_and_dummies[["count"]]

# Calcul des Y = log(Y + 1)
Y_train['log_count'] = Y_train['count'].map(lambda x: np.log1p(x))
Y_train = Y_train.drop("count", axis = 1)

/home/osboxes/anaconda3/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [75]:
X_train.shape

(10886, 6429)

In [76]:
features_names = list(X_train)
new_features_names = [re.sub("\[|\]|\<|\>",'',line) for line in features_names]
X_train.columns = new_features_names

features_names = list(X_test)
new_features_names = [re.sub("\[|\]|\<|\>",'',line) for line in features_names]
X_test.columns = new_features_names

In [77]:
params = {
    "colsample_bytree": uniform(0.3, 0.7),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.4, 0.6),  # c'est quoi ?
    "reg_alpha": uniform(0, 0.05)
}

# n_iter = 200 avant
search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, n_iter=85, cv=3, verbose=1, n_jobs=2, return_train_score=True)

search.fit(X_train, Y_train)

Fitting 3 folds for each of 85 candidates, totalling 255 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: 